In [1]:
#@title Parameters
#MATH 691Y - Project
#Simulated annealing demo with TSP

import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
from itertools import islice
import copy
from scipy.stats import norm
from math import sqrt,log,sin,cos,pi,exp

# Number of simulation repetitions
num_rep = 10#@param {type:"number"}

# Number of trucks
num_trucks = 40#@param {type: "number"}

# SA parameters
init_temp = 100#@param {type:"number"}
final_temp = 1#@param {type:"number"}
cooling = 0.99#@param {type:"number"}

# Location labels
depot = [1]
forests = [2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,23,24,25,26,29,31,32,33,34,35,36,37,40,41,42,44,45,47,48,50,52,53,55,57,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,74,76,78,79]
sawmills = [7,20,21,22,27,28,30,38,39,43,46,49,51,54,56,68,75,77]

# Orders to be fulfilled and x,y coordinates of each location
all_orders0 = [(3,46),(58,43),(69,49),(58,46),(58,43),(58,43),(58,43),(58,43),(58,43),(16,49),(79,43),(45,46),(58,46),(42,75),(47,49),(58,46),(58,43),(58,43),(58,27),(78,51),(58,27),(37,7),(37,51),(14,68),(58,43),(18,49),(32,30),(69,51),(47,39),(79,7),(12,7),(47,39),(42,75),(37,7),(10,49),(5,54),(72,75),(58,43),(69,49),(79,43),(44,75),(24,49),(47,39),(25,56),(19,49),(58,27),(58,43),(19,49),(58,27),(41,21),(50,49),(57,27),(18,49),(69,7),(34,7),(69,51),(12,75),(5,56),(69,7),(69,49),(9,43),(79,46),(33,51),(60,46),(69,51),(52,54),(34,51),(24,49),(9,43),(12,7),(60,7),(69,51),(66,7),(58,43),(10,49),(41,21),(64,27),(48,27),(19,39),(34,51),(61,27),(79,7),(58,43),(2,7),(61,27),(5,54),(35,51),(79,56),(2,49),(23,49),(35,51),(3,7),(19,39),(19,39),(58,43),(42,75),(57,27),(37,7),(4,43),(59,49),(61,27),(79,7),(24,7),(67,7),(9,22),(45,46),(24,49),(3,51),(3,7),(61,27),(69,49),(19,49),(19,49),(60,7),(12,7),(58,43),(69,51),(79,46),(79,7),(79,43),(19,39),(31,49),(18,75),(65,7),(61,49),(69,7),(18,7),(37,7),(67,49),(47,49),(40,49),(45,39),(40,49),(13,51),(24,7),(36,46),(35,51),(79,7),(44,75),(34,51),(6,75),(69,51),(34,51),(37,43),(11,75),(69,51),(5,54),(6,75),(12,7),(10,46),(35,51),(58,43),(12,7),(47,39),(63,46),(53,77),(45,46),(12,7),(12,7),(69,49),(55,7),(65,51),(64,51),(79,46),(37,7),(37,51),(18,49),(64,27),(18,7),(18,49),(52,54),(55,56),(47,39),(12,75),(6,75),(23,27),(24,7),(58,43),(69,51),(67,7),(10,49),(79,56),(79,43),(5,54),(58,43),(3,43),(55,7),(57,27),(64,27),(44,75),(47,39),(64,27),(45,39),(16,49),(45,39),(70,39),(19,49),(10,49),(34,51),(70,39),(65,51),(66,43),(62,56),(73,49),(79,7),(2,49),(70,39),(70,39),(19,49),(52,54),(3,7),(12,7),(69,51),(44,75),(6,75),(44,75),(13,51),(19,39),(55,56),(76,46),(67,7),(67,49),(74,49),(5,54),(12,75),(12,7),(23,20),(60,7),(12,7),(19,49),(2,7),(71,7),(35,51),(24,49),(64,51),(24,49),(79,43),(64,49),(57,27),(18,49),(69,51),(79,46),(79,7),(29,49),(19,39),(52,54),(47,49),(50,49),(8,51),(25,46),(79,7),(12,7),(36,56),(2,7),(48,43),(48,20),(9,22),(2,49),(61,75),(76,20),(79,46),(64,51),(5,54),(25,77),(79,7),(57,27),(2,7),(23,49),(64,27),(13,51),(13,38),(2,7),(17,51),(61,27),(3,51),(63,46),(23,49),(23,20),(58,46),(58,43),(23,27),(2,49),(37,7),(15,56),(65,51),(61,75),(18,7),(58,46),(47,39),(71,28),(35,51),(2,7),(69,49),(58,43),(8,51),(12,7),(26,49),(58,27),(45,49),(65,51)];
# xcoord = [301389,235965,269650,294745,255950,244150,233462,258730,304350,339360,280625,230785,358950,339032,295125,237780,309555,263350,302610,320101,300405,354079,255250,233830,287225,308950,233442,253303,338290,322803,254650,330815,294150,259150,235335,273410,326855,433531,300420,379295,273835,248050,312013,255895,329865,311553,361455,237150,286232,235885,337480,237061,317500,231069,233945,286967,259150,215280,266050,253288,224050,275262,237550,230450,253485,255750,256350,283885,233280,364470,220630,233150,370335,234085,254635,226457,319865,227745,291506]
# ycoord = [422458,401250,408350,413515,415750,424450,365355,434490,390950,392673,412620,368490,391950,422171,430915,413070,500075,441350,393430,434187,421645,474447,433050,412935,400140,377050,377693,397325,475990,429689,408450,485915,343950,434650,416455,386810,386535,457150,468789,419890,413200,382550,414862,400615,398530,414516,401135,423350,438535,412155,440577,375291,514500,368866,404340,401910,409350,440260,437050,397320,443950,402503,421450,419650,409795,434950,433750,430747,417250,395685,420600,446450,509560,408635,378348,427498,532410,425985,364500]

all_orders2 = [(13,7),(13,7),(13,7),(27,53),(37,7),(74,42),(74,42),(59,7),(36,22),(69,55),(18,53),(3,7),(50,42),(50,53),(70,7),(10,53),(3,7),(70,46),(73,7),(59,58),(84,46),(3,7),(69,55),(81,22),(57,82),(83,29),(62,46),(74,42),(56,58),(73,7),(62,46),(70,46),(13,7),(62,46),(20,42),(11,49),(62,46),(13,80),(62,49),(69,7),(84,49),(76,22),(14,49),(40,55),(16,53),(28,7),(3,49),(13,7),(3,7),(70,7),(69,55),(56,58),(70,7),(13,7),(62,46),(70,7),(3,7),(68,32),(20,42),(50,42),(30,82),(16,53),(13,7),(18,7),(39,29),(74,42),(13,7),(64,7),(43,53),(37,55),(62,46),(13,7),(38,55),(48,42),(20,53),(59,60),(13,7),(38,55),(11,53),(48,49),(73,53),(84,7),(62,46),(57,23),(5,60),(18,7),(64,7),(10,53),(84,49),(10,49),(69,7),(13,7),(45,80),(84,60),(62,32),(74,42),(84,46),(40,55),(43,53),(13,7),(45,80),(13,7),(18,53),(73,53),(50,42),(8,55),(84,7),(40,7),(57,55),(13,7),(65,32),(69,55),(40,7),(13,80),(11,53),(9,46),(68,53),(20,53),(43,53),(6,80),(68,55),(41,53),(59,7),(71,53),(71,7),(40,55),(84,7),(77,53),(77,53),(68,32),(28,7),(30,49),(30,60),(50,42),(73,53),(18,53),(74,42),(84,7),(5,58),(67,49),(75,7),(68,32),(31,72),(26,53),(73,55),(5,58),(84,49),(13,7),(74,42),(13,7),(59,7),(84,60),(73,55),(78,53),(34,53),(40,55),(30,24),(48,42),(73,55),(36,55),(73,7),(10,53),(61,22),(10,53),(35,22),(70,7),(2,53),(18,53),(62,32),(2,7),(40,7),(20,53),(40,55),(73,53),(79,53),(38,55),(57,82),(76,22),(17,55),(77,53),(84,7),(62,32),(57,82),(13,80),(34,53),(61,32),(75,7),(18,53),(18,53),(73,7),(13,7),(27,53),(62,46),(62,49),(65,32),(31,53),(28,53),(13,7),(4,53),(73,55),(13,7),(9,46),(2,53),(61,32),(64,7),(84,49),(16,53),(57,82),(81,7),(62,32),(62,32),(63,53),(62,46),(18,53),(64,49),(14,55),(6,80),(74,42),(74,42),(40,7),(64,7),(5,60),(64,7),(48,53),(51,53),(3,7),(54,53),(3,7),(62,46),(12,80),(56,58),(2,7),(40,7),(19,53),(2,7),(37,55),(81,49),(30,49),(84,49),(67,49),(13,7),(40,7),(44,24),(84,7),(76,80),(50,42),(71,7),(68,32),(14,49),(28,7),(77,53),(84,7),(13,7),(40,7),(62,46),(28,7),(45,80),(26,53),(76,32),(13,7),(38,55),(65,32),(2,7),(64,7),(73,55),(63,53),(84,7),(52,32),(40,7),(47,80),(2,53),(62,46),(20,42),(12,80),(2,53),(62,46),(74,42),(9,25),(56,58),(18,80),(62,46),(5,58),(84,7),(74,42),(65,32),(18,53),(38,55),(16,53),(56,58),(69,55),(48,53),(5,58),(13,80),(15,60),(50,53),(14,55),(64,7),(67,49),(13,7),(73,53),(50,53),(21,25),(15,60),(84,49),(20,42),(71,53),(57,82),(27,53),(48,53),(54,53),(64,53),(27,53),(66,60),(51,53),(57,55),(64,7),(28,53),(52,22),(28,53),(57,55),(73,55),(28,53),(73,53),(54,53),(34,53),(8,7),(73,53),(57,82),(36,22),(27,53),(33,53),(18,7),(52,49),(8,7),(52,46),(66,60),(81,7),(18,53),(84,7),(64,7),(81,7),(40,49),(8,55),(65,32),(68,32),(68,32),(27,53),(37,55),(69,55),(61,32)]
# xcoord2 = [301389,235965,269650,304465,255950,244150,233462,258730,304350,339360,254280,280625,230785,358950,295125,237780,309555,263350,367535,302610,284945,320101,432365,300405,354079,349500,255250,233830,308195,287225,308950,233442,277090,338290,216312,294150,259150,235335,264745,326855,236145,300420,379295,273835,248050,312013,255895,329865,311553,361455,239175,237150,286232,235885,337480,237061,317500,231069,233945,286967,259150,215280,266050,253288,224050,275262,237550,230450,253485,255750,256350,324420,233280,364470,220630,233150,370335,359890,234085,254635,226457,319865,227832,291506]
# ycoord2 = [422458,401250,408350,393960,415750,424450,365355,434490,390950,392673,431400,412620,368490,391950,430915,413070,500075,441350,419755,393430,438504,434187,419270,421645,474447,422500,433050,412935,422358,400140,377050,377693,366340,475990,440203,343950,434650,416455,435580,386535,429575,468789,419890,413200,382550,414862,400615,398530,414516,401135,454170,423350,438535,412155,440577,375291,514500,368866,404340,401910,409350,440260,437050,397320,443950,402503,421450,419650,409795,434950,433750,425120,417250,395685,420600,446450,509560,408715,408635,378348,427498,532410,433083,364500]

# xcoord3 = [301389,368422,237150,354079,275262,283685,233945,336598,254280,228432,215280,237780,233830,259150,331200,280380,300420,237115,307850,329119,228782,370335,234085,322803,224050,282168,361455,280625,233150,255895,256350,253288,230450,258730,300405,263350,235447,269650,226982,312013,255250,357560,394613,319865,376955,264745,339360,337480,308950,304350,233280,233607,364630,226457,338290,255750,250500,235885,253485,237550,242350,285273,433850,230785,255950,317500,364470,294150,233442,267445,311928,248050,247935,432365,326855,325708,235965,358950,312965,339937,320101,249050,263650,285932,226050,254635,328530,269675,338110,286967,360705,314970,359890,331387,302610,308195,291506,230950]
# ycoord3 = [422458,389225,423350,474447,402503,406725,404340,398875,431400,402705,440260,413070,412935,409350,367200,411095,468789,408630,413059,432364,401618,509560,408635,429689,443950,415585,401135,412620,446450,400615,433750,397320,419650,434490,421645,441350,335013,408350,407105,414862,433050,523082,435309,532410,402500,435580,392673,440577,377050,390950,417250,365019,388075,427498,475990,434950,358300,412155,409795,421450,407050,438419,457638,368490,415750,514500,395685,343950,377693,417395,414407,382550,452190,419270,386535,405392,401250,391950,478472,385625,434187,404150,436850,437990,422450,378348,387633,276595,433925,401910,400470,468660,408715,367648,393430,422358,364500,368818]
all_orders3 = [(95,17),(14,69),(95,17),(14,71),(14,69),(49,17),(80,90),(68,48),(28,86),(95,17),(95,84),(49,84),(80,90),(27,17),(37,71),(27,17),(18,48),(14,69),(82,71),(80,35),(70,48),(19,48),(36,86),(34,84),(26,48),(26,48),(37,71),(95,84),(95,84),(77,84),(11,40),(80,90),(50,40),(26,52),(59,84),(66,89),(49,84),(34,84),(78,71),(6,71),(15,71),(60,52),(31,84),(23,48),(94,71),(26,48),(32,52),(80,90),(93,84),(80,90),(41,84),(61,98),(36,52),(25,20),(80,35),(49,84),(32,52),(75,48),(41,71),(28,86),(57,84),(25,69),(97,52),(70,48),(39,52),(95,84),(23,48),(8,48),(9,71),(34,48),(77,84),(67,17),(26,71),(94,71),(80,90),(67,17),(13,84),(61,69),(27,17),(15,71),(78,71),(10,71),(49,71),(12,52),(49,84),(8,71),(77,52),(3,71),(64,86),(70,52),(76,48),(94,71),(77,86),(77,86),(39,52),(39,52),(80,90),(80,90),(57,71),(65,98),(70,48),(8,48),(3,71),(95,84),(2,71),(85,86),(76,48),(3,69),(15,71),(6,48),(15,71),(75,48),(78,71),(26,52),(10,71),(57,86),(80,90),(85,86),(28,86),(78,71),(39,52),(78,74),(41,71),(41,84),(38,48),(10,71),(34,84),(39,48),(77,84),(58,84),(12,52),(77,84),(2,71),(57,71),(39,52),(97,52),(18,48),(10,52),(8,71),(67,17),(45,84),(10,71),(62,4),(14,86),(13,52),(31,52),(76,48),(56,52),(13,52),(80,90),(87,81),(15,52),(77,84),(11,40),(18,48),(57,52),(79,48),(57,52),(60,71),(25,69),(10,52),(70,48),(70,48),(33,84),(25,69),(10,52),(12,52),(57,52),(8,52),(22,84),(95,84),(87,81),(53,71),(49,84),(68,48),(57,52),(77,52),(49,17),(53,71),(78,71),(82,98),(18,48),(45,84),(27,17),(64,52),(91,17),(10,52),(87,81),(57,52),(30,98),(45,84),(67,17),(29,71),(28,86),(13,84),(13,84),(78,48),(78,81),(25,40),(39,52),(8,48),(77,84),(80,35),(32,84),(37,71),(72,71),(16,48),(72,98),(8,71),(39,52),(46,84),(55,84),(39,52),(72,86),(77,52),(11,69),(15,52),(57,86),(53,71),(36,52),(87,52),(75,52),(80,96),(93,84),(80,90),(41,84),(80,35),(26,48),(97,52),(67,17),(53,48),(39,52),(68,48),(39,52),(10,71),(54,81),(82,98),(31,48),(91,17),(75,71),(12,84),(12,84),(75,52),(11,40),(68,48),(25,69),(32,71),(26,71),(30,86),(76,48),(39,52),(78,74),(7,52),(66,44),(53,71),(51,52),(53,71),(26,71),(65,98),(10,52),(39,52),(41,84),(8,71),(10,52),(26,48),(10,71),(64,86),(78,71),(76,48),(80,90),(70,48),(12,52),(92,48),(57,52),(41,69),(79,42),(26,81),(30,86),(70,84),(56,52),(53,48),(39,71),(88,98),(51,52),(10,71),(15,71),(76,48),(58,86),(61,98),(57,71),(39,52),(80,90),(53,48),(23,48),(76,48),(36,52),(61,98),(26,52),(10,52),(75,52),(18,48),(67,17),(76,48),(37,71),(77,86),(65,98),(57,52),(80,35),(65,98),(58,86),(12,52),(60,52),(75,40),(13,84),(80,35),(59,84),(39,52),(66,48),(10,71),(15,71),(8,48),(53,48),(27,17),(88,98),(45,84),(10,71),(75,48),(16,71),(36,52),(18,48),(66,89),(79,42),(5,71),(87,71),(78,81),(8,43),(13,84),(78,48),(12,52),(80,90),(27,17),(30,86),(18,48),(75,48),(87,52),(68,24),(67,17),(38,52),(7,52),(67,17),(15,71),(26,71),(15,71),(23,48),(3,69),(83,84),(41,84),(79,42),(78,63),(64,52),(34,84),(12,52),(8,48),(11,40),(47,43),(97,52),(25,69),(94,71),(37,71),(12,52),(41,73),(11,69),(7,52),(41,84),(32,52),(82,98),(61,69),(10,98),(14,81),(11,40),(77,52),(78,71),(41,71),(23,48),(12,84),(8,48),(7,52),(18,48),(72,98),(10,52),(8,48),(61,69),(26,71),(11,69),(16,48),(70,48),(34,48),(21,84),(26,52),(70,52)]

# xcoord4 = [301389,368422,237150,274006,275262,233945,336598,228432,215280,377700,237780,254650,233830,259150,331200,280380,300420,227745,329119,228782,370335,349500,234085,224050,331525,282168,361455,233150,255895,256350,253288,230450,258730,300405,263350,235447,269650,226982,233619,312013,255250,357560,376955,339360,337480,244150,308950,304350,233280,233607,364630,338290,255750,250500,235885,253485,237550,242350,433850,230785,255950,317500,364470,294150,233442,273550,267445,311928,248050,247935,432365,326855,325708,235965,298705,358950,339937,320101,273410,263650,285932,226050,254635,328530,234350,269675,286967,359890,331387,302610,308195,291506,230950]
# ycoord4 = [422458,389225,423350,345995,402503,404340,398875,402705,440260,423250,413070,408450,412935,409350,367200,411095,468789,425985,432364,401618,509560,422500,408635,443950,514445,415585,401135,446450,400615,433750,397320,419650,434490,421645,441350,335013,408350,407105,365925,414862,433050,523082,402500,392673,440577,424450,377050,390950,417250,365019,388075,475990,434950,358300,412155,409795,421450,407050,457638,368490,415750,514500,395685,343950,377693,437650,417395,414407,382550,452190,419270,386535,405392,401250,361730,391950,385625,434187,386810,436850,437990,422450,378348,387633,410050,276595,401910,408715,367648,393430,422358,364500,368818]
all_orders4 = [(47,81),(11,50),(56,81),(38,68),(47,17),(56,81),(60,50),(11,50),(74,81),(38,50),(60,50),(92,68),(11,50),(8,68),(15,68),(57,50),(54,81),(47,81),(35,50),(74,81),(8,68),(9,40),(74,83),(74,81),(38,50),(54,68),(47,81),(33,81),(35,50),(74,50),(35,50),(36,68),(41,81),(55,81),(67,50),(64,45),(30,50),(21,81),(14,68),(77,34),(4,39),(41,68),(8,50),(38,50),(30,50),(11,50),(38,50),(3,68),(23,45),(51,68),(77,45),(63,17),(35,81),(52,81),(51,68),(3,68),(15,68),(67,45),(77,34),(43,81),(41,81),(47,68),(9,40),(8,50),(47,81),(31,50),(77,87),(7,45),(74,50),(47,81),(9,40),(74,81),(14,65),(9,65),(92,50),(8,50),(38,68),(38,50),(49,50),(27,17),(90,17),(9,40),(53,50),(13,50),(33,81),(76,68),(47,17),(76,68),(41,81),(88,81),(33,81),(20,81),(60,50),(33,45),(49,50),(33,81),(80,81),(14,65),(67,45),(37,50),(41,81),(35,50),(41,68),(89,68),(28,68),(76,45),(38,50),(90,81),(46,81),(13,81),(90,81),(35,81),(74,81),(67,45),(41,65),(90,81),(44,81),(74,50),(89,68),(8,50),(9,40),(90,17),(67,50),(7,68),(41,81),(26,45),(8,50),(90,17),(24,65),(21,81),(77,87),(12,81),(6,50),(9,65),(69,68),(77,34),(26,50),(36,68),(49,81),(11,50),(26,68),(90,81),(13,50),(26,45),(38,50),(90,81),(13,81),(92,40),(41,81),(3,65),(38,50),(77,68),(29,93),(3,70),(26,45),(6,50),(72,50),(9,65),(30,50),(77,87),(77,34),(14,83),(77,34),(26,68),(85,81),(84,50),(54,50),(21,81),(27,81),(36,68),(24,65),(11,50),(2,68),(41,81),(38,50),(24,65),(23,45),(31,68),(90,81),(46,81),(86,93),(51,68),(88,81),(11,50),(38,50),(49,50),(31,50),(61,93),(72,68),(38,50),(38,50),(74,50),(14,65),(8,93),(11,50),(10,81),(57,34),(8,50),(24,40),(27,81),(18,81),(38,50),(55,83),(24,65),(76,45),(41,81),(41,81),(88,81),(22,81),(9,19),(30,50),(26,68),(36,68),(51,68),(27,17),(67,45),(76,71),(73,45),(33,81),(9,40),(26,68),(41,65),(9,65),(60,50),(51,68),(58,65),(30,81),(77,34),(84,50),(54,68),(77,87),(2,68),(72,50),(11,81),(77,91),(90,81),(24,40),(10,81),(13,50),(90,81),(41,70),(37,50),(8,68),(77,87),(85,81),(38,50),(38,50),(33,45),(67,45),(73,45),(90,81),(89,68),(72,40),(48,40),(66,34),(24,65),(49,81),(38,50),(41,81),(8,68),(11,50),(27,17),(54,50),(27,17),(24,65),(77,87),(54,50),(31,50),(37,40),(11,50),(77,87),(77,87),(38,50),(54,50),(12,81),(33,45),(33,81),(15,50),(55,81),(7,45),(7,68),(41,81),(49,81),(24,65),(74,81),(63,17),(54,50),(16,68),(25,42),(72,45),(51,45),(67,45),(89,68),(63,17),(63,17),(77,87),(72,50),(52,81),(90,17),(11,81),(9,40),(72,68),(27,17),(21,81),(38,50),(26,50),(49,81),(33,81),(41,65),(44,81),(30,50),(15,50),(92,50),(8,68),(5,81),(82,83),(54,50),(28,65),(11,50),(74,83),(77,87),(8,50),(26,50),(79,34),(92,50),(76,78),(67,45),(16,34),(77,87),(26,68),(21,81),(38,50),(23,45),(60,50),(74,81),(8,50),(58,93),(32,81),(27,17),(57,68),(38,50),(61,93),(57,50),(28,65),(41,81),(77,87),(67,50),(54,68),(26,45),(8,68),(61,93),(41,81),(54,81),(74,81),(8,50),(24,81),(7,45),(26,68),(35,81),(92,81),(77,87),(28,65),(58,65),(35,81),(77,34),(15,68),(35,50),(43,81),(8,50),(41,68),(90,81),(27,17),(38,50),(31,81),(8,68),(92,50),(41,81),(77,87),(67,45),(63,17),(8,50),(62,45),(38,50),(8,50),(26,45),(54,83),(27,81),(77,87),(90,17),(16,45),(11,50),(38,50),(90,17),(74,50),(38,50),(9,65),(54,83),(31,50),(57,50),(54,50),(36,91),(4,75),(28,68),(69,68),(76,59),(67,68),(38,50),(53,40),(13,81),(4,40),(41,68),(9,40),(77,87),(63,17),(8,50),(77,34),(54,50),(62,45),(55,81),(6,50)]

# xcoord5 = [301389,368422,237150,216312,233945,336598,228432,215280,338392,237780,254650,233830,259150,331200,280380,300420,237115,307850,329119,228782,370335,234085,322803,224050,331525,282168,361455,280625,302270,233150,255895,256350,236145,253288,258730,300405,263350,235447,269650,226982,312013,255250,357560,319865,376955,339360,337480,244150,308950,304350,233280,233607,364630,338290,255750,250500,235885,253485,237550,242350,285273,230785,255950,317500,364470,294150,233442,236030,273550,267445,311928,248050,247935,432365,326855,325708,235965,358950,339937,320101,263650,285932,226050,254635,328530,234350,269675,338110,286967,359890,331387,302610,308195,291506,230950]
# ycoord5 = [422458,389225,423350,440203,404340,398875,402705,440260,501305,413070,408450,412935,409350,367200,411095,468789,408630,413059,432364,401618,509560,408635,429689,443950,514445,415585,401135,412620,412845,446450,400615,433750,429575,397320,434490,421645,441350,335013,408350,407105,414862,433050,523082,532410,402500,392673,440577,424450,377050,390950,417250,365019,388075,475990,434950,358300,412155,409795,421450,407050,438419,368490,415750,514500,395685,343950,377693,404525,437650,417395,414407,382550,452190,419270,386535,405392,401250,391950,385625,434187,436850,437990,422450,378348,387633,410050,276595,433925,401910,408715,367648,393430,422358,364500,368818]
all_orders5 = [(33,41),(92,16),(64,44),(26,52),(64,88),(8,41),(79,71),(33,41),(10,52),(85,52),(10,82),(37,52),(7,71),(53,71),(34,52),(53,71),(53,71),(69,36),(64,44),(26,47),(53,71),(49,82),(12,52),(64,47),(77,82),(26,71),(56,52),(70,93),(77,82),(56,71),(77,52),(22,47),(56,82),(92,16),(3,67),(38,71),(70,47),(79,36),(59,52),(53,71),(22,47),(58,82),(13,71),(70,93),(40,52),(40,71),(26,52),(77,82),(76,47),(79,36),(85,71),(8,41),(75,52),(58,82),(8,41),(26,47),(26,71),(75,47),(40,52),(27,16),(85,80),(94,52),(56,52),(72,71),(40,52),(79,89),(85,80),(10,52),(70,47),(77,82),(42,82),(91,71),(92,82),(42,67),(63,95),(12,82),(56,52),(37,52),(6,71),(62,52),(92,82),(42,82),(10,52),(3,71),(33,41),(56,82),(61,29),(65,16),(22,47),(3,71),(78,71),(3,67),(92,82),(56,71),(57,82),(49,16),(3,73),(37,52),(26,52),(40,52),(40,52),(92,16),(53,71),(62,52),(92,82),(26,71),(77,84),(64,88),(56,52),(64,88),(79,36),(56,71),(21,82),(37,82),(3,67),(3,80),(13,67),(2,71),(62,52),(56,82),(87,95),(55,52),(24,67),(59,71),(7,52),(54,82),(75,41),(70,52),(22,47),(76,47),(22,47),(10,52),(24,67),(38,71),(77,82),(7,52),(17,47),(92,82),(91,71),(17,47),(76,47),(14,71),(77,52),(75,52),(40,52),(24,80),(40,52),(34,52),(51,52),(10,52),(62,84),(55,52),(64,88),(91,71),(12,52),(46,19),(8,67),(31,84),(21,82),(63,95),(40,52),(77,52),(6,71),(45,82),(7,71),(75,71),(78,71),(77,82),(30,67),(3,67),(94,52),(8,19),(78,80),(42,71),(53,71),(70,47),(49,82),(39,52),(72,71),(22,47),(7,52),(35,82),(64,88),(56,84),(35,82),(62,84),(65,16),(40,52),(56,52),(78,47),(22,47),(90,82),(76,47),(38,52),(42,82),(59,52),(92,82),(56,84),(9,23),(42,71),(39,52),(24,41),(40,52),(27,16),(53,47),(7,52),(79,36),(51,52),(79,36),(76,47),(64,88),(64,23),(35,47),(35,82),(94,52),(26,71),(56,52),(22,47),(63,95),(37,52),(3,71),(92,16),(72,95),(85,52),(40,52),(38,71),(53,47),(75,71),(10,52),(78,71),(40,52),(86,82),(79,89),(14,52),(94,52),(78,74),(7,71),(56,52),(85,52),(42,82),(62,52),(53,71),(7,71),(6,47),(34,71),(77,52),(15,71),(72,84),(5,52),(81,82),(92,82),(40,52),(64,44),(56,52),(42,82),(56,52),(20,82),(27,16),(48,82),(70,52),(91,71),(7,52),(25,43),(10,52),(12,52),(75,71),(6,47),(6,71),(10,52),(42,82),(42,71),(70,47),(5,52),(70,47),(50,41),(53,47),(6,52),(79,89),(65,16),(12,82),(75,52),(37,84),(78,47),(7,71),(30,71),(7,52),(79,36),(5,52),(91,71),(65,16),(37,52),(77,52),(32,52),(42,82),(38,71),(78,71),(26,71),(10,52),(22,47),(56,71),(70,71),(40,52),(79,71),(26,71),(10,52),(40,52),(86,82),(42,82),(12,52),(7,71),(8,41),(12,82),(42,82),(94,41),(70,47),(27,16),(35,82),(14,71),(8,41),(40,52),(40,52),(62,84),(6,71),(30,67),(6,47),(6,47),(91,71),(6,47),(8,67),(32,52),(30,71),(34,52),(14,71),(39,52),(7,52),(6,71),(6,71),(28,84),(55,41),(53,71),(51,47),(26,47),(8,41),(68,84),(38,71),(14,52),(51,82),(78,71),(62,84),(56,52),(37,84),(2,71),(24,67),(51,52),(70,47),(46,82),(14,71),(35,47),(4,19),(18,47),(64,44),(94,52),(22,47),(17,47),(70,52),(24,67),(66,47),(56,71),(49,82),(27,16),(26,47),(20,82),(6,47),(79,36),(56,84),(94,52),(10,82),(7,52),(53,47),(70,52),(35,82),(70,47),(70,82),(57,84),(78,71),(3,67),(27,16),(55,41),(70,52),(40,52),(14,52),(92,16),(7,52),(7,71),(85,80),(40,52),(72,84),(10,52),(60,95),(62,52),(6,71),(83,84),(56,52),(56,84),(59,52),(17,47),(7,95),(7,52),(42,82),(56,84),(32,82),(40,52),(53,71),(11,82),(63,95),(53,71),(7,71),(3,67),(51,82),(24,67)]

xcoord = [301389,368422,237150,216312,275262,233945,336598,228432,215280,338392,237780,233830,259150,331200,280380,300420,237115,329119,228782,370335,234085,322803,224050,331525,282168,361455,280625,233150,255895,256350,253288,258730,300405,263350,235447,269650,226982,312013,255250,357560,394613,376955,337752,339360,337480,244150,304350,233280,233607,364630,226457,338290,255750,250500,235885,295125,253485,237550,242350,433850,230785,255950,317500,364470,294150,233442,273550,267445,311928,289855,248050,247935,432365,326855,325708,235965,358950,339937,320101,273410,263650,285932,226050,254635,328530,269675,338110,286967,359890,331387,302610,308195,291506,230950]
ycoord = [422458,389225,423350,440203,402503,404340,398875,402705,440260,501305,413070,412935,409350,367200,411095,468789,408630,432364,401618,509560,408635,429689,443950,514445,415585,401135,412620,446450,400615,433750,397320,434490,421645,441350,335013,408350,407105,414862,433050,523082,435309,402500,500198,392673,440577,424450,390950,417250,365019,388075,427498,475990,434950,358300,412155,430915,409795,421450,407050,457638,368490,415750,514500,395685,343950,377693,437650,417395,414407,403005,382550,452190,419270,386535,405392,401250,391950,385625,434187,386810,436850,437990,422450,378348,387633,276595,433925,401910,408715,367648,393430,422358,364500,368818]
all_orders = [(78,88),(93,49),(93,49),(25,69),(78,33),(58,49),(78,33),(93,49),(37,49),(51,79),(59,94),(85,79),(78,33),(51,79),(37,49),(7,69),(25,45),(35,69),(71,94),(58,69),(62,94),(13,69),(37,49),(34,49),(71,84),(54,82),(85,79),(3,69),(61,49),(78,33),(6,49),(34,49),(71,69),(58,49),(13,84),(93,49),(7,69),(76,84),(23,79),(78,88),(76,82),(77,79),(58,49),(78,88),(93,49),(8,49),(83,84),(57,82),(3,69),(31,49),(25,69),(48,49),(43,22),(61,49),(23,79),(77,73),(53,49),(25,45),(29,94),(78,88),(77,69),(68,79),(12,49),(12,82),(34,49),(7,45),(10,22),(90,69),(61,84),(78,88),(37,49),(90,69),(7,45),(68,45),(76,49),(61,49),(54,69),(8,69),(3,66),(76,49),(57,82),(48,49),(68,49),(11,49),(86,94),(58,69),(63,87),(68,49),(27,84),(78,92),(54,82),(39,69),(76,82),(37,49),(93,49),(25,69),(86,94),(23,79),(34,82),(74,49),(36,49),(59,94),(25,49),(6,49),(13,66),(58,49),(58,49),(78,33),(34,49),(78,88),(77,79),(78,88),(78,33),(68,49),(50,69),(7,69),(7,45),(77,69),(37,49),(8,94),(29,94),(39,82),(78,79),(25,69),(3,66),(78,88),(77,69),(68,45),(50,69),(44,18),(3,66),(13,66),(37,49),(74,69),(56,41),(12,49),(67,33),(39,82),(62,94),(50,69),(25,69),(31,49),(68,45),(62,94),(6,49),(8,49),(8,49),(78,88),(91,16),(85,79),(11,49),(54,84),(78,88),(74,49),(61,84),(39,82),(55,82),(75,45),(12,82),(54,49),(20,82),(26,16),(68,79),(7,45),(21,45),(85,79),(50,69),(62,94),(35,69),(50,69),(50,45),(8,49),(11,49),(11,49),(9,18),(91,16),(37,49),(11,49),(93,49),(30,49),(85,49),(62,94),(37,49),(28,66),(30,49),(39,82),(61,49),(68,45),(93,82),(25,69),(75,45),(62,94),(36,49),(78,88),(91,82),(54,49),(15,45),(86,94),(12,49),(59,66),(68,45),(9,38),(54,49),(9,66),(37,49),(59,94),(26,16),(68,79),(90,69),(68,92),(75,45),(50,69),(44,69),(68,45),(29,84),(55,84),(54,69),(48,49),(65,45),(13,79),(59,94),(46,82),(65,45),(2,69),(44,41),(52,82),(23,66),(6,49),(8,49),(74,49),(14,49),(39,82),(77,45),(50,45),(37,49),(68,49),(90,69),(8,69),(8,49),(76,82),(23,92),(37,49),(65,22),(39,69),(27,84),(54,49),(29,84),(76,82),(34,49),(31,49),(39,66),(54,49),(8,49),(13,66),(21,45),(39,66),(15,69),(35,92),(37,49),(9,38),(35,69),(8,69),(15,45),(37,49),(8,49),(23,38),(24,40),(17,45),(37,49),(54,84),(78,88),(17,45),(29,94),(9,38),(32,45),(68,92),(13,69),(34,82),(8,69),(93,49),(8,69),(54,84),(25,45),(12,82),(50,69),(58,49),(34,84),(37,69),(78,88),(11,49),(50,45),(68,69),(44,82),(25,69),(54,49),(59,94),(50,69),(30,49),(54,69),(48,82),(78,92),(3,72),(54,84),(8,69),(93,49),(3,79),(25,49),(78,88),(75,45),(14,49),(39,82),(31,69),(50,69),(3,69),(54,49),(15,33),(76,49),(42,82),(78,33),(29,94),(7,41),(54,49),(23,66),(4,18),(68,45),(8,49),(37,49),(78,88),(54,49),(15,45),(7,69),(44,41),(90,69),(68,45),(78,70),(11,49),(36,49),(78,88),(3,66),(32,82),(54,49),(26,82),(68,45),(78,69),(25,79),(30,49),(71,94),(78,69),(76,82),(28,69),(7,69),(65,45),(78,88),(61,49),(34,49),(5,49),(7,45),(44,18),(76,49),(85,79),(3,66),(25,45),(17,45),(37,49),(14,69),(35,92),(37,49),(34,49),(28,66),(29,84),(80,33),(77,60),(7,69),(74,49),(86,94),(30,49),(75,69),(54,69),(25,69),(74,38),(37,49),(54,49),(89,82),(76,49),(14,49),(23,66),(8,69),(37,49),(8,49),(23,79),(59,66),(13,69),(37,49),(61,84),(71,69),(68,45),(12,82),(62,94),(11,49),(14,49),(64,16),(62,94),(48,82),(47,38),(63,45),(50,69),(25,49),(14,49),(91,82),(68,45),(36,49),(81,82),(11,82),(9,38),(26,82),(68,45),(7,69),(14,69),(76,49),(35,49),(37,49),(26,16),(55,84),(68,45),(74,45),(77,69),(85,49),(25,79),(78,88),(25,22),(78,88),(54,49),(26,16),(77,69),(19,82),(39,72),(23,18),(93,49),(2,69),(37,49)]


### Create fleet of trucks

In [2]:
# Singular truck object
class Truck:
  def __init__(self,number):
    self.orders = list() # list of orders that truck fulfills
    self.route = list()  # complete truck route including starting and ending at hub
    self.id = number # truck number
    self.distance = 0 # distance of truck route

  def set_distance(self):
    if self.distance != 0:
      self.distance = 0
    d = 0.0
    for i in range(len(self.route)-1):
      d += np.sqrt ((ycoord[self.route[i]-1]-ycoord[self.route[i+1]-1])**2 + (xcoord[self.route[i]-1]-xcoord[self.route[i+1]-1])**2)
    self.distance = d

  def get_distance(self):
    return self.distance

  def set_route(self):
    self.route = list()
    self.route.append(1)
    for i in range(len(self.orders)):
        for j in range(2):
            self.route.append(self.orders[i][j])
    self.route.append(1)

  def update(self,orders):
    self.orders = copy.deepcopy(orders)
    self.set_route()
    self.set_distance()

  def get_route(self):
    return self.route

  def get_orders(self):
    return self.orders


In [3]:
# Fleet of trucks
class Fleet:
  def __init__(self,num_trucks):
    self.total_dist = 0 # total distance travelled by fleet of trucks
    self.curr_orders = list() # orders to be fulfilled
    self.trucks = list() # list of Trucks in fleet
    self.num_trucks = num_trucks # number of trucks

    # initialize Truck objects
    for i in range(self.num_trucks):
      self.trucks.append(Truck(i+1))

  # Split orders equally among trucks
  def assign_equal_orders(self,orders):
    split_list = split(orders,self.num_trucks)
    for i in range(self.num_trucks):
      self.trucks[i].update(split_list[i])
      self.curr_orders.append(split_list[i])

  # Assign orders according to optimized distance
  def assign_orders(self,orders):
    self.curr_orders = []
    for i in range(self.num_trucks):
      self.trucks[i].update(orders[i])
      self.curr_orders.append(orders[i])

  def get_trucks(self):
    return self.trucks

  def get_distance(self):
    self.total_dist = 0
    for i in range(self.num_trucks):
      self.total_dist += self.trucks[i].get_distance()
    return self.total_dist

  def get_curr_orders(self):
    return self.curr_orders

  def get_num_trucks(self):
    return self.num_trucks


### Helper functions

In [4]:
def split(all_orders,num_trucks):
  k,m = divmod(len(all_orders),num_trucks)
  orders_gen = (all_orders[i*k+min(i,m):(i+1)*k+min(i+1,m)] for i in range(num_trucks))
  split_list = list()
  for n in orders_gen:
    split_list.append(n)
  return split_list

def shuffle(all_orders):
  copied_all_orders = all_orders.copy()
  np.random.shuffle(copied_all_orders)
  return copied_all_orders

def swap(curr_sol):
  copied_curr_sol = copy.deepcopy(curr_sol)
  trucki = random.randrange(0,num_trucks,1)
  truckj = random.randrange(0,num_trucks,1)
  if trucki == truckj:
    orderi = random.randrange(0,len(copied_curr_sol[trucki]),1)
    orderj = random.randrange(0,len(copied_curr_sol[trucki]),1)
    while orderi == orderj:
      orderi = random.randrange(0,len(copied_curr_sol[trucki]),1)
      orderj = random.randrange(0,len(copied_curr_sol[trucki]),1)

    temp = copied_curr_sol[trucki][orderi]
    copied_curr_sol[trucki][orderi] = copied_curr_sol[trucki][orderj]
    copied_curr_sol[trucki][orderj] = temp
  else:
    orderi = random.randrange(0,len(copied_curr_sol[trucki]),1)
    orderj = random.randrange(0,len(copied_curr_sol[truckj]),1)
    temp = copied_curr_sol[trucki][orderi]
    copied_curr_sol[trucki][orderi] = copied_curr_sol[truckj][orderj]
    copied_curr_sol[truckj][orderj] = temp

  return copied_curr_sol

def transfer(curr_sol):
  copied_curr_sol = copy.deepcopy(curr_sol)
  trucki = random.randrange(0,num_trucks,1)
  truckj = random.randrange(0,num_trucks,1)
  while trucki == truckj:
    trucki = random.randrange(0,num_trucks,1)
    truckj = random.randrange(0,num_trucks,1)

  if (len(copied_curr_sol[trucki]) > 1):
    orderi = random.randrange(0,len(copied_curr_sol[trucki]),1)
    copied_curr_sol[truckj].append(copied_curr_sol[trucki][orderi])
    copied_curr_sol[trucki].pop(orderi)
  return copied_curr_sol


In [5]:
"""
Plots graph of a truck route

Parameters
----------
route: list
   route of a singular truck
xcoord: list
    x-coordinates of all locations in numerical order
    0 (hub), 1 (logger), 2 (mill)... 0 (hub)
ycoord: list
    y-coordinates of all locations in numerical order
    0 (hub), 1 (logger), 2 (mill)... 0 (hub)

"""
def plot_graph(Fleet,xcoord,ycoord):

    # locations = ["hub","logger1","mill2","logger3","mill4","logger5","mill6","logger7","mill8", "logger9","mill10","hub"]
    plt.scatter(xcoord,ycoord)
    # for i, label in enumerate(locations):
    #     plt.annotate(label,(xcoord[i],ycoord[i]))

    xcoordnew = list()
    ycoordnew = list()

    for i in range(Fleet.get_num_trucks()):
      route = Fleet.get_trucks()[i].get_route()
      for i in range(len(route)):
          xcoordnew.append(xcoord[route[i]-1])
          ycoordnew.append(ycoord[route[i]-1])

    plt.plot(xcoordnew,ycoordnew)


### Define Statistical Estimator Model


In [6]:
### Code from Peter J Haas from UMASS CSCI 550 Simulation course ###
class Estimator:
    """ Computes point estimates and confidence intervals """
    def __init__(self, conf_pct, epsilon, relative):
        self.epsilon = epsilon # precison of CI
        self.relative = relative # relative or absolute precision
        self.k = 0  # number of values processed so far
        self.sum = 0.0  # running sum of values
        self.v = 0.0  # running value of (k-1)*variance
        self.z = norm.ppf(0.5 + (conf_pct / 200))  # CI normal quantile
        self.conf_str = str(conf_pct) + "%"  # string of form "xx%" for xx% CI

    def reset(self):
        self.k = 0
        self.sum = 0
        self.v = 0

    def process_next_val(self, value):
        self.k += 1
        if self.k > 1:
            diff = self.sum - (self.k - 1) * value
            self.v += diff/self.k * diff/(self.k-1)
        self.sum += value

    def get_variance(self):
        return self.v/(self.k-1) if self.k > 1 else 0

    def get_mean(self):
        return self.sum/self.k if self.k >= 1 else 0

    def get_conf_interval(self):
        hw = self.z * sqrt(self.get_variance()/self.k)
        point_est = self.get_mean()
        c_low = point_est - hw
        c_high = point_est + hw
        rstr = "{0} Confidence Interval: [ {1:.4f}, {2:.4f} ]  (hw = {3:.4f})"
        return rstr.format(self.conf_str, c_low, c_high, hw)

    def get_num_trials(self):
        var = self.get_variance()
        if var == 0:
            return "UNAVAILABLE (Need Var > 0)"
        if self.relative:
            width = self.get_mean() * self.epsilon
            rstr = "Est. # of reps for +/- {0:.2f}% accuracy: {1}"
            acc = 100 * self.epsilon
        else:
            width = self.epsilon
            rstr = "Est. # of reps for +/- {0:.4f} accuracy: {1}"
            acc = self.epsilon
        nreps = int(1+(var * self.z * self.z)/(width * width))

        return rstr.format(acc, nreps)

### Simulation Model

In [7]:
### Altered code from Peter J Haas from UMASS CSCI 550 Simulation course ###
class Sim_Model:
  def __init__(self):
    pass

  def do_rep(self,est,max_iter,init_temp,xcoord,ycoord):
    curr_temp = init_temp

    # Generate initial solution
    shuffled_orders = shuffle(all_orders)
    fleet = Fleet(num_trucks)
    fleet.assign_equal_orders(shuffled_orders)
    curr_sol = fleet.get_curr_orders()
    curr_dist = fleet.get_distance()
    # print("{0:>20}{1:>15,.2f}".format("Initial distance: ", fleet.get_distance()))

    # Simulated Annealing
    while curr_temp > final_temp:
      for i in range(max_iter):
        # Generate adjacent solution by swapping two orders
        swapped_orders = swap(curr_sol)
        fleet.assign_orders(swapped_orders)
        swap_dist = fleet.get_distance()

        # Accept the next sol as the current solution
        if (fleet.get_distance()) < (curr_dist):
            curr_dist = fleet.get_distance()
            curr_sol = fleet.get_curr_orders()
        # O/w accept the next sol with certain probability p
        else:
            delta = ((fleet.get_distance()) - (curr_dist))
            if np.random.uniform(0,1) < (curr_temp/(curr_temp + delta)): # accept worse sol
              curr_dist = fleet.get_distance()
              curr_sol = fleet.get_curr_orders()
            else: # don't accept the worse sol
              fleet.assign_orders(curr_sol)

      curr_temp = curr_temp * cooling

    perf_meas = fleet.get_distance()
    est.process_next_val(perf_meas)
    # plot_graph(fleet,xcoord,ycoord)
    # print("{0:>20}{1:>15,.2f}".format("Final distance:",fleet.get_distance()))

    return perf_meas


In [8]:
est = Estimator(95,0.05,relative =True)#95% CI
sim = Sim_Model()

best = 41000000000
for rep in range(int(num_rep)):
  val = sim.do_rep(est,5,init_temp,xcoord,ycoord)
  if val < best:
    best = val

print("{0:>20}{1:>15,.2f}".format("Best distance:",best))
print("{0:>20}{1:>15,.2f}".format("Average distance:",est.get_mean()))
print("{0:>20}{1:>15,.2f}".format("Variance:",est.get_variance()))



      Best distance:  45,888,179.72
   Average distance:  46,171,338.53
           Variance:69,938,413,144.14
